In [27]:
import gspread
import nest_asyncio
from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime

In [28]:
nest_asyncio.apply()
s = HTMLSession()

In [29]:
url = 'https://www.bseindia.com/markets/MarketInfo/NoticesCirculars.aspx'
r = s.get(url)
r.html.arender(sleep=5)

<coroutine object HTML.arender at 0x0000018E79A75FE0>

In [30]:

if r.status_code == 200:
    df = pd.DataFrame(columns=['Notice_Number','Subject','Segment Name','Category Name','Department'])
    if r.status_code == 200:
        table = r.html.xpath('//*[@id="divmain"]/div[4]/div/div',first = True)
        soup = bs(table.html,'lxml')
        table = soup.find('table', {'id': 'ContentPlaceHolder1_GridView1'})
        Master_Data = []
        if table:
            # Extract data from the table
            rows = table.find_all('tr')
            for row in rows[1:]:  # Skip the header row (index 0)
                columns = row.find_all(['td'])
                row = [tr.text for tr in columns]
                filter_row = row[:5]
                l = len(df)
                df.loc[l] = filter_row
        else:
            print(r)
else:
    print(r.status_code)

In [31]:
Notifications_Links = []
table = r.html.xpath('//*[@id="divmain"]/div[4]/div/div',first = True)
soup = bs(table.html,'lxml')
table = soup.find('table', {'id': 'ContentPlaceHolder1_GridView1'})
links = table.find_all('a',{'class': 'tablebluelink'})
for href_l in links:
    href_p = href_l.get('href')
    Notifications_Links.append("www.bseindia.com"+href_p)

In [32]:
df['Notification_link'] = Notifications_Links
df.head(50)

,Notice_Number,Subject,Segment Name,Category Name,Department,Notification_link
0,20240206-33,Listing of Equity Shares of Gabriel Pet Straps...,SME,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
1,20240206-32,Listing of Equity Shares of Jasch Industries L...,Equity,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
2,20240206-31,Listing of New Securities of LKP Securities Li...,Equity,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
3,20240206-30,Listing of new Commercial Paper of Nuvama Weal...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
4,20240206-29,Listing of new Commercial Paper of Bajaj Finan...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
5,20240206-28,Listing of new Commercial Paper of ICICI Secur...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
6,20240206-27,Listing of new Commercial Paper of Birla Group...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
7,20240206-26,Listing of new Commercial Paper of Tata Realty...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
8,20240206-25,Listing of new Commercial Paper of SEIL Energy...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
9,20240206-24,Listing of new Commercial Paper of ICICI HOME ...,Debt,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...


In [33]:
gc = gspread.service_account(filename='creds.json')
spreadsheet_name = 'Notifications and Listings'
sheet_name = 'BSE_Forthcoming_listing'
sh = gc.open(spreadsheet_name).worksheet(sheet_name)


In [34]:
if r.status_code == 200:
    existing_data_range = sh.range('A2:F' + str(sh.row_count))
    for cell in existing_data_range:
        cell.value = ''

    sh.update_cells(existing_data_range)
else :
    pass

In [35]:
"""if r.status_code == 200:
        Notice_Link = []
        for row in rows[1:]:  # Skip the header row (index 0)
                links = row.find_all('a')
                for link in links:
                        notice_download = link.get('href')
                        with_header = "https://www.bseindia.com"+notice_download 
                        Notice_Link.append(with_header)  

        df["Notice_Link"] = Notice_Link  
else:
 pass"""

'if r.status_code == 200:\n        Notice_Link = []\n        for row in rows[1:]:  # Skip the header row (index 0)\n                links = row.find_all(\'a\')\n                for link in links:\n                        notice_download = link.get(\'href\')\n                        with_header = "https://www.bseindia.com"+notice_download \n                        Notice_Link.append(with_header)  \n\n        df["Notice_Link"] = Notice_Link  \nelse:\n pass'

In [36]:
Filtered_df = df[df ['Subject'].str.contains('Listing of Equity|Listing of Units')& df['Category Name'].str.contains('Company related')]


,Notice_Number,Subject,Segment Name,Category Name,Department,Notification_link
0,20240206-33,Listing of Equity Shares of Gabriel Pet Straps...,SME,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
1,20240206-32,Listing of Equity Shares of Jasch Industries L...,Equity,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
14,20240206-19,Listing of Units of HDFC NIFTY PSU BANK ETF a...,Mutual Fund,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
15,20240206-18,Listing of Units of DSP Nifty Healthcare ETF a...,Mutual Fund,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
20,20240206-13,Listing of Equity Shares of Dugar Housing Deve...,Equity,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...
28,20240206-5,Listing of Equity Shares of Gabriel Pet Straps...,SME,Company related,Listing Operations,www.bseindia.com/markets/MarketInfo/DispNewNot...


In [40]:
if r.status_code == 200:
    values_lists = []
    df_dict = Filtered_df.to_dict(orient='records')
    for data in df_dict:
        values_lists.append([str(data['Notice_Number']), str(data['Subject']), str(data['Segment Name']),str(data['Category Name']),str(data['Notification_link'])])
    sh.append_row([f'Batch ran at: {datetime.datetime.now()}'])
    sh.append_rows(values_lists)
else :
    pass

In [38]:
#filtered_df1 = df[df['Department'].str.contains('Listing') & df['Subject'].str.contains('Listing of Equity') ]
#filtered_df1.head(50)

In [39]:
#for i in filtered_df1['Notice_Link']:
    #print(i)